<a href="https://colab.research.google.com/github/hemanthh35/smarthealth/blob/main/Dna_Sequencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the generated data
data = pd.read_csv('/content/synthetic_dna_data_extended.csv')

# Split the features (X) and the target (y)
X = data.drop('DiseaseRisk', axis=1)
y = data['DiseaseRisk']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
from xgboost import XGBClassifier
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Save the model
import joblib
joblib.dump(model, 'xgboost_model.pkl')


Model Accuracy: 100.00%


['xgboost_model.pkl']

In [ ]:
import joblib

# Load the trained model
model = joblib.load('xgboost_model.pkl')


In [ ]:
!pip install gradio xgboost scikit-learn pandas joblib matplotlib


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=fa02baf29c005c57e67fe428be36f3bb4703f2b900f4e124d1110317f394d757
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import io
from fpdf import FPDF
import joblib

# Load the trained model (make sure you've trained and saved it before)
model = joblib.load('xgboost_model.pkl')

# Disease mapping (adjust to your diseases)
disease_mapping = {
    0: "Low Risk Disease",
    1: "Medium Risk Disease",
    2: "High Risk Disease"
}

# Function to get disease explanation
def get_disease_explanation(disease_class):
    explanations = {
        0: "Low risk disease is typically associated with no significant genetic predispositions.",
        1: "Medium risk disease has a moderate genetic link but lifestyle factors play a major role.",
        2: "High risk disease indicates a strong genetic predisposition, and preventive measures are crucial."
    }
    return explanations.get(disease_class, "No explanation available.")

# Function to get disease recommendations
def get_recommendations(disease_class):
    recommendations = {
        0: "Low-Risk Diseases:Maintain a balanced diet rich in vegetables, fruits, and whole grains.Engage in at least 30 minutes of moderate exercise, such as walking, every day.Get 7-8 hours of sleep every night.Stay hydrated by drinking plenty of water.Avoid smoking and limit alcohol consumption.Keep stress levels low through activities like meditation, yoga, or mindfulness.Schedule regular health check-ups to monitor general well-being.",
        1: " Moderate-Risk Diseases:Prioritize heart-healthy foods like lean proteins, healthy fats (olive oil, nuts), and fiber-rich grains.Include cardiovascular exercises such as jogging, cycling, or swimming 3-5 times a week.Maintain a healthy weight and monitor blood pressure and cholesterol levels.Limit processed foods, reduce salt intake, and minimize sugar consumption.Focus on stress management through relaxation techniques and hobbies.Ensure adequate sleep, aiming for 7-8 hours each night.Stay consistent with health screenings and follow-ups with your healthcare provider.",
        2: "High-Risk Diseases:Follow a plant-based or Mediterranean diet, focusing on antioxidants, anti-inflammatory foods, and omega-3 fatty acids.Increase physical activity by aiming for 5 days of moderate to vigorous exercise, such as brisk walking, running, or strength training.Regularly check key health markers like blood glucose, blood pressure, and cholesterol.Minimize stress through relaxation practices, therapy, or social support.Consider genetic counseling and early screenings for targeted prevention.Consult a healthcare provider about possible medical interventions, medications, or supplements.Avoid environmental toxins and prioritize a clean and healthy living space.Stay consistent with your healthcare plan and communicate with your doctor about any concerns."
    }
    return recommendations.get(disease_class, "No recommendations available.")

# Function to generate PDF report with patient info and test results
def generate_pdf(patient_details, snp_values, predicted_class, confidence, disease_name, explanation, recommendations):
    # Create PDF report with patient info
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(200, 10, txt="Disease Risk Prediction Report", ln=True, align='C')

    pdf.ln(10)
    pdf.set_font('Arial', '', 12)
    pdf.cell(200, 10, txt=f"Patient Name: {patient_details['name']}", ln=True)
    pdf.cell(200, 10, txt=f"Age: {patient_details['age']}", ln=True)
    pdf.cell(200, 10, txt=f"Gender: {patient_details['gender']}", ln=True)

    pdf.ln(5)
    pdf.cell(200, 10, txt=f"Blood Test Results: {patient_details['blood_test']}", ln=True)

    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Predicted Disease: {disease_name}", ln=True)
    pdf.cell(200, 10, txt=f"Confidence: {confidence*100:.2f}%", ln=True)

    pdf.ln(5)
    pdf.multi_cell(0, 10, f"Explanation: {explanation}")
    pdf.multi_cell(0, 10, f"Recommendations: {recommendations}")

    file_path = "/content/disease_risk_report.pdf"
    pdf.output(file_path)

    return file_path

# Main prediction function
def predict_disease(patient_name, age, gender, blood_test, input_values):
    try:
        # 1. Store patient details
        patient_details = {
            "name": patient_name,
            "age": age,
            "gender": gender,
            "blood_test": blood_test
        }

        # 2. Parse SNP input values into a list of integers
        snp_values = [int(x) for x in input_values.split(',')]
        snp_array = np.array(snp_values).reshape(1, -1)

        # 3. Predict probabilities
        probabilities = model.predict_proba(snp_array)[0]

        # 4. Get predicted class
        predicted_class = np.argmax(probabilities)
        predicted_confidence = probabilities[predicted_class]

        # 5. Map class index to disease name
        predicted_disease = disease_mapping[predicted_class]

        # 6. Generate explanation and recommendations
        explanation = get_disease_explanation(predicted_class)
        recommendations = get_recommendations(predicted_class)

        # 7. Generate result with highlighting
        if predicted_confidence >= 0.8:
            result = f"## ⚠️ **High Risk Detected!**\n\n**Predicted Disease:** {predicted_disease}\n\n**Confidence:** `{predicted_confidence*100:.2f}%`"
        else:
            result = f"## ✅ **Prediction Result**\n\n**Predicted Disease:** {predicted_disease}\n\n**Confidence:** `{predicted_confidence*100:.2f}%`"

        # 8. Generate PDF report with patient details
        pdf_file_path = generate_pdf(patient_details, snp_values, predicted_class, predicted_confidence, predicted_disease, explanation, recommendations)

        return result, pdf_file_path

    except Exception as e:
        return f"❌ Error: {str(e)}", None

# Build Gradio UI with a form for patient details and SNP values input
interface = gr.Interface(
    fn=predict_disease,
    inputs=[
        gr.Textbox(placeholder="Enter patient's full name", label="Patient Name"),
        gr.Textbox(placeholder="Enter patient's age", label="Age"),  # Changed to Textbox for age input
        gr.Dropdown(["Male", "Female", "Other"], label="Gender"),
        gr.Textbox(placeholder="Enter blood test results (e.g., Blood Pressure: 120/80, Cholesterol: 180)", label="Blood Test Results"),
        gr.Textbox(placeholder="Enter SNP values like: 0,1,2,1,0,1,2,0,1,0", label="SNP Input (Comma-Separated)")
    ],
    outputs=[
        gr.Markdown(label="Prediction Result"),
        gr.File(label="Download PDF Report")
    ],
    title="🧬 Disease Risk Prediction with Patient Details",
    description="Enter patient details and SNP values to predict disease risk and generate a detailed report."
)

# Run the Gradio interface
interface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://87ca56cb8569d7220e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
